In [7]:
'''
FMG特征未进行单独归一化
代码有问题，不是最终使用版本
'''


import joblib
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, roc_curve, auc
from sklearn.feature_selection import RFE

from utils_logger import logger_init
import logging

from utils_model import form_params_grid

In [2]:
"""
准备数据集
"""
data_df = pd.read_csv(r"E:\Data\paper2\iFEMG\iFEMG_curl_abs_normed_all.csv", index_col = None, header = 0)

# data_df = pd.concat([data_df1, data_df2], axis = 0, ignore_index = True)
# data_df = data_df2
print(data_df.shape)
data_df.head()

(608, 29)


,subject_info_subject_name,subject_info_height,subject_info_weight,subject_info_age,subject_info_label,bicps_br_ave,bicps_br_std,bicps_br_initial_pressure_min,bicps_br_initial_pressure_ave,bicps_br_act_ave_list,...,tricps_br_medial_rst_ave_list,tricps_br_medial_rst_std_list,tricps_br_lateral_ave,tricps_br_lateral_std,tricps_br_lateral_initial_pressure_min,tricps_br_lateral_initial_pressure_ave,tricps_br_lateral_act_ave_list,tricps_br_lateral_act_std_list,tricps_br_lateral_rst_ave_list,tricps_br_lateral_rst_std_list
0,hpy,172,80,23,0.0,1913.122977,427.01095,1460,1491.491151,2231.203862,...,800.548480,9.065329,877.422752,158.26857,716,735.225231,1021.484658,17.566674,735.225231,5.179396
1,hpy,172,80,23,0.0,1913.122977,427.01095,1460,1491.491151,2315.715081,...,794.071135,12.130896,877.422752,158.26857,716,735.225231,1046.557646,16.481837,731.395349,6.892843
2,hpy,172,80,23,0.0,1913.122977,427.01095,1460,1491.491151,2338.333283,...,816.399265,8.103951,877.422752,158.26857,716,735.225231,1059.129322,18.271512,756.904317,7.884648
3,hpy,172,80,23,0.0,1913.122977,427.01095,1460,1491.491151,2397.877389,...,774.606084,11.059522,877.422752,158.26857,716,735.225231,1057.169393,18.868385,737.398961,8.895111
4,hpy,172,80,23,0.0,1913.122977,427.01095,1460,1491.491151,2396.461597,...,743.107683,45.345220,877.422752,158.26857,716,735.225231,1063.527714,15.643251,720.428593,12.365997


In [3]:
"""
数据清洗 remove the data bellow:
    1. NaN value
    2. label is MVC
"""
# replace 'MVC' with NaN
data_df = data_df.replace('MVC', np.nan)
# delete NaN value
data_df = data_df.dropna(how = 'any')

print(data_df.shape)
data_df.dtypes

(608, 29)


subject_info_subject_name                  object
subject_info_height                         int64
subject_info_weight                         int64
subject_info_age                            int64
subject_info_label                        float64
bicps_br_ave                              float64
bicps_br_std                              float64
bicps_br_initial_pressure_min               int64
bicps_br_initial_pressure_ave             float64
bicps_br_act_ave_list                     float64
bicps_br_act_std_list                     float64
bicps_br_rst_ave_list                     float64
bicps_br_rst_std_list                     float64
tricps_br_medial_ave                      float64
tricps_br_medial_std                      float64
tricps_br_medial_initial_pressure_min       int64
tricps_br_medial_initial_pressure_ave     float64
tricps_br_medial_act_ave_list             float64
tricps_br_medial_act_std_list             float64
tricps_br_medial_rst_ave_list             float64


In [4]:
logger_init(log_file_name="FMG model")

In [6]:
"""
随机划分训练集SVR模型
"""

columns_to_scale = ['subject_info_height',
                    'subject_info_weight',
                    'subject_info_age',    
                    'bicps_br_initial_pressure_ave',
                    'bicps_br_act_ave_list',
                    'tricps_br_medial_initial_pressure_ave',
                    'tricps_br_medial_act_ave_list',
                    'tricps_br_lateral_initial_pressure_ave',
                    'tricps_br_lateral_act_ave_list']

y_data = data_df.loc[:, 'subject_info_label'].values
x_data = data_df.loc[:, columns_to_scale].values

print(x_data.shape)
print(y_data.shape)

# 模型训练
train_data_r, test_data_r, train_label, test_label = train_test_split(x_data, y_data, test_size = 0.2, random_state = 0)
# train_data = preprocessing.normalize(train_data_r, norm = 'max')
# test_data = preprocessing.normalize(test_data_r, norm = 'max')
scaler = preprocessing.StandardScaler().fit(train_data_r)
train_data = scaler.transform(train_data_r)
test_data = scaler.transform(test_data_r)


"""
Grid searching best parameters of SVR model
"""
# 记录开始训练时间
start_time = time.perf_counter()

# 自动选择合适的参数
svr = GridSearchCV(SVR(), param_grid={"kernel": ("linear", "rbf"), "C": np.logspace(-3, 3, 7), "gamma": np.logspace(-3, 3, 7)}, n_jobs=-1)
svr.fit(train_data, train_label)
print(f"Best params: {svr.best_params_}")

end_time = time.perf_counter()
print(f"training time(min): {(end_time - start_time)/60}")

score_test = svr.score(test_data, test_label)
score_train = svr.score(train_data, train_label)
print(f"train score: {str(score_train)}")
print(f"test score: {str(score_test)}")
test_pre = svr.predict(test_data)
train_pre = svr.predict(train_data)
print(f"train mean squared error: {mean_squared_error(train_label, train_pre)}")
print(f"test mean squared error: {mean_squared_error(test_label, test_pre)}")
    


Best params: {'C': 1000.0, 'gamma': 1.0, 'kernel': 'rbf'}
training time(min): 0.2833533700000165
train score: 0.9906777622060253
test score: 0.9788034346531146
train mean squared error: 0.005240640769027414
test mean squared error: 0.011374782757787586


In [9]:
# 每次取一个受试者作为测试集，其余人的数据作为训练集

# 选取数据集名称
columns_to_scale = ['subject_info_height',
                    'subject_info_weight',
                    'subject_info_age',    
                    'bicps_br_initial_pressure_ave',
                    'bicps_br_act_ave_list',
                    'tricps_br_medial_initial_pressure_ave',
                    'tricps_br_medial_act_ave_list',
                    'tricps_br_lateral_initial_pressure_ave',
                    'tricps_br_lateral_act_ave_list']

for subject in set(data_df["subject_info_subject_name"]):
    logging.info(f"=======================================================")
    logging.info(f"test subject: {subject}")
    test_df = data_df[data_df["subject_info_subject_name"] == subject]  # 测试集
    train_df = data_df[data_df["subject_info_subject_name"] != subject] # 训练集
    x_test = test_df.loc[:, columns_to_scale].values
    y_test = test_df.loc[:, 'subject_info_label'].values
    x_train = train_df.loc[:, columns_to_scale].values
    y_train = train_df.loc[:, 'subject_info_label'].values
    logging.info(f"x_test: {x_test.shape}")
    logging.info(f"y_test: {y_test.shape}")
    logging.info(f"x_train: {x_train.shape}")
    logging.info(f"y_train: {y_train.shape}")
    
    # 数据标准化
    scaler = preprocessing.StandardScaler().fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    """
    Grid searching best parameters of SVR model
    """
    # 记录开始训练时间
    start_time = time.perf_counter()

    # 自动选择合适的参数
    svr = GridSearchCV(SVR(), param_grid={"kernel": ("linear", "rbf"), "C": np.logspace(-3, 3, 7), "gamma": np.logspace(-3, 3, 7)}, n_jobs=-1)
    svr.fit(x_train_scaled, y_train)
    logging.info(f"Best params: {svr.best_params_}")

    end_time = time.perf_counter()
    logging.info(f"training time(min): {(end_time - start_time)/60}")

    score_test = svr.score(x_test_scaled, y_test)
    score_train = svr.score(x_train_scaled, y_train)
    logging.info(f"train score: {str(score_train)}")
    logging.info(f"test score: {str(score_test)}")
    test_pre = svr.predict(x_test_scaled)
    train_pre = svr.predict(x_train_scaled)
    logging.info(f"train mean squared error: {mean_squared_error(y_train, train_pre)}")
    logging.info(f"test mean squared error: {mean_squared_error(y_test, test_pre)}")
    logging.info(f"true label: {y_test}")
    logging.info(f"predicted label: {test_pre}")
    logging.info(f"true - predicted: {y_test - test_pre}")
    pass